In [ ]:
### This code is used to accept tenders and then onload the positions and try to get the maximum profit out of it
import requests
import collections

API_KEY = {'X-API-key': 'XXXXXXX'}

### this function gets the bid and the ask of the stock given to it
def ticker_bid_ask(ticker):
     with requests.Session() as s:
        s.headers.update(API_KEY)
        payload = {'ticker':ticker}
        resp = s.get('http://localhost:9999/v1/securities/book', params = payload)
        if resp.ok:
            book = resp.json()
            return book['bids'][0]['price'],book['asks'][0]['price']
        raise ApiException("Authoriation error. Please check API key")

        
### this code is to place a market order to buy
def buying(ticker,quantity):
    with requests.Session() as s:
        s.headers.update(API_KEY)
        mkt_buy_params = {'ticker': ticker, 'type': 'MARKET', 'quantity': quantity,
                          'action': 'BUY'}
        resp = s.post('http://localhost:9999/v1/orders', params=mkt_buy_params)
        if resp.ok:
            mkt_order = resp.json()
            id = mkt_order['order_id']
            #print('The market buy order was submitted and has ID', id)
        else:
            print('The order was not successfully submitted!')
            
### this code is to place a market order to sell
def selling(ticker,quantity):
    with requests.Session() as s:
        s.headers.update(API_KEY)
        lmt_sell_params = {'ticker': ticker, 'type': 'MARKET', 'quantity': quantity,
        #'price': 10.00,
        'action': 'SELL'}
        resp = s.post('http://localhost:9999/v1/orders', params=lmt_sell_params)
        if resp.ok:
            lmt_order = resp.json()
            id = lmt_order['order_id']
            #print('The limit sell order was submitted and has ID', id)
        else:
            print('The order was not successfully submitted!')
            
### this function gets the current ticker of the case
def tick():
    with requests.Session() as s: # step 3
         s.headers.update(API_KEY) # step 4
         resp = s.get('http://localhost:9999/v1/case')
         if resp.ok: 
             case = resp.json()
             tick = case['tick'] # accessing the 'tick' value that was returned
             #print('The case is on tick', tick) # step 8 
             return tick
            
### this funciton is to get all the active tenders
def get_tenders():
    with requests.Session() as s: 
         s.headers.update(API_KEY) 
         resp = s.get('http://localhost:9999/v1/tenders') 
         if resp.ok:
             tenders = resp.json() 
             tender_info = tenders
             return tender_info

### this is to accept the tenders
def accept_tender(ID):
    with requests.Session() as s: 
         s.headers.update(API_KEY) 
         resp = s.post('http://localhost:9999/v1/tenders/{}'.format(ID)) 
        
### to decline the tenders
def decline_tender(ID):
    with requests.Session() as s: 
         s.headers.update(API_KEY) 
         resp = s.delete('http://localhost:9999/v1/tenders/{}'.format(ID)) 
        
### this is to get the bid book data from a ticker
def securities_data_bid(ticker):
    with requests.Session() as s: # step 3
         s.headers.update(API_KEY) # step 4
         payload = {'ticker':ticker,'limit':50}
         resp = s.get('http://localhost:9999/v1/securities/book',params=payload) # step 5
         if resp.ok: # step 6
             bid_info = resp.json() # step 7
             bid_info = bid_info['bids'] # accessing the 'tick' value that was returned
             #print('The case is on tick', tick) # step 8 
             return bid_info
        
### this is to get the average price to fullfill the order of X amount at market price considering both markets (A nd M)

def avg_price_ask_bid_combine(ticker,quant,bid_ask):
    if bid_ask == 'ask':
        bid_ask='asks'
    else:
        bid_ask='bids'
    with requests.Session() as s: # step 3
        s.headers.update(API_KEY) # step 4
        ticker_A=both_tickers2(ticker)[0]
        payload = {'ticker':ticker_A,'limit':50}
        resp_A = s.get('http://localhost:9999/v1/securities/book',params=payload) # step 5
        if resp_A.ok: # step 6
             ask_info_A = resp_A.json() # step 7
             ask_info_A = ask_info_A[bid_ask] # accessing the 'tick' value that was returned
         
        ticker_M=both_tickers2(ticker)[1]
        payload = {'ticker':ticker_M,'limit':50}
        resp_M = s.get('http://localhost:9999/v1/securities/book',params=payload) # step 5
        if resp_M.ok: # step 6
             ask_info_M = resp_M.json() # step 7
             ask_info_M = ask_info_M[bid_ask] # accessing the 'tick' value that was returned
         
        for x in range(0,len(ask_info_M)):
             ask_info_A.append(ask_info_M[x])
        sorted_list=sorted(ask_info_A, key = lambda i: i['price'])
        
        pos=sorted_list
        position_close = 0
        x = -1
        price = 0
        price_info = []
        while position_close < quant and x+2 <= len(pos):
            x = x + 1
            pos_data = [pos[x]['ticker'],pos[x]['quantity'],pos[x]['price'],pos[x]['quantity_filled']]
            PosData = collections.namedtuple("PosData", ["ticker","quantity","price","quantity_filled"])
            pos_data=PosData(pos_data[0],pos_data[1],pos_data[2],pos_data[3])
            position_order = pos_data.quantity - pos_data.quantity_filled
            if position_close + position_order <= quant:
                position_close = position_close + position_order
                price_info.append([position_order,pos_data.price,position_order/quant])
            elif position_close + position_order > quant:
                #price = price+ ((position - position_close)  * pos_data.price)
                price_info.append([(quant - position_close),pos_data.price,(quant - position_close)/quant])
                position_close = position_close +(quant - position_close) 
        avg_price=0
        if position_close == quant:
            for x in range(0,len(price_info)):
                avg_price= avg_price + (price_info[x][1]*price_info[x][2])
            return(avg_price)
        else:
            if bid_ask =='asks':
                avg_price = -1000000
                return(avg_price)
            else:
                avg_price = 100000
                return(avg_price)
##this funcitons are to get the main ticker and then get the both variatons: example  CRZY_M -> CRZY -> CRZY_M CRZY_A
### the difference between the functions is the type of data they need, both_tickers2 needs a single ticker
### and both_tickers needs the info as it is cleaned in the main code
def both_tickers(t_data):
    single_ticker=t_data[0]['ticker'].split("_")[0]
    input_list = [[single_ticker,"_","A"],[single_ticker,"_","M"]]
    joined = [''.join(row) for row in input_list]
    return joined
def both_tickers2(t_data):
    single_ticker=t_data.split("_")[0]
    input_list = [[single_ticker,"_","A"],[single_ticker,"_","M"]]
    joined = [''.join(row) for row in input_list]
    return joined




#################### THE COOOOODEEEEE

########## PHASE 2
import requests
import collections
import time

API_KEY = {'X-API-key': 'XXXXXXX'}
treshold = 0.2
tick2=0
unload_qty = 250
with requests.Session() as s:
    while tick()<299:
        time.sleep(0.05)
        tick1=tick()
        if(tick1%10==0 and tick2!=tick1):
            print('Tick',tick1)
        tick2=tick1
            ## get the tenders information and slice the info into a list with usesable names
        t_data =get_tenders() ###erase t_data2
        if len(t_data)!=0:
            for x in range(0,len(t_data)):
                tender_data = [t_data[x]['tender_id'],t_data[x]['expires'],t_data[x]['ticker'],t_data[x]['quantity'],t_data[x]['action']
                                   ,t_data[x]['is_fixed_bid'],t_data[x]['price']]
                TenderData = collections.namedtuple("TenderData", ["tender_id", "expires", "ticker", "quantity", "action", "is_fixed_bid", "price"])
                tender_data=TenderData(tender_data[0],tender_data[1],tender_data[2],tender_data[3],tender_data[4],tender_data[5],tender_data[6]) 
                two_tickers=both_tickers(t_data)
                #print(tender_data)
                #### here stops the tender info management

                #### here we start the decisions to buy/sell, the next code is for tenders with a specific price
                if tender_data.is_fixed_bid == True:
                    if tender_data.action == "SELL": #BID - In this part I am selling to the company therefore I need to buy to close the position
                        if(avg_price_ask_bid_combine(tender_data.ticker, tender_data.quantity,'bid') + treshold) <= tender_data.price: #if the parameters meet then we buy
                                                                    ##and loop through the position until we sell everything
                            accept_tender(tender_data.tender_id)
                            close_pos = 0
                            stop = False
                            while close_pos != tender_data.quantity and stop == False:
                                try:
                                    price_A=ticker_bid_ask(two_tickers[0])
                                    price_M=ticker_bid_ask(two_tickers[1])
                                    bid_A = price_A[0]
                                    bid_M = price_M[0]
                                    if bid_A<bid_M:
                                        winner_ticker = two_tickers[0]
                                    else:
                                        winner_ticker = two_tickers[1]

                                    if close_pos + unload_qty <= tender_data.quantity:
                                        buying(winner_ticker, unload_qty)
                                        close_pos = close_pos + unload_qty
                                        #sleep(sleeping)
                                    else:
                                        buying(winner_ticker, (tender_data.quantity - close_pos))
                                        close_pos = close_pos + (tender_data.quantity - close_pos)
                                        stop = True
                                except:
                                    if close_pos + unload_qty <= tender_data.quantity:
                                        buying(winner_ticker, unload_qty)
                                        close_pos = close_pos + unload_qty
                                        #sleep(sleeping)
                                    else:
                                        buying(winner_ticker, (tender_data.quantity - close_pos))
                                        close_pos = close_pos + (tender_data.quantity - close_pos)
                                        stop = True
                        else:
                            decline_tender(tender_data.tender_id)

                    elif tender_data.action == "BUY":  #ASK - In this part I am selling to the company therefore I need to sell to close the position
                        if(avg_price_ask_bid_combine(tender_data.ticker,tender_data.quantity,'ask')-treshold) >= tender_data.price:
                            accept_tender(tender_data.tender_id)
                            close_pos = 0
                            stop = False
                            while close_pos != tender_data.quantity and stop == False:
                                try:
                                    price_A=ticker_bid_ask(two_tickers[0])
                                    price_M=ticker_bid_ask(two_tickers[1])
                                    ask_A = price_A[1]
                                    ask_M = price_M[1]
                                    if ask_A>ask_M:
                                        winner_ticker = two_tickers[0]
                                    else:
                                        winner_ticker = two_tickers[1]

                                    if close_pos + unload_qty <= tender_data.quantity:
                                        selling(winner_ticker, unload_qty)
                                        close_pos = close_pos + unload_qty
                                        #sleep(sleeping)

                                    else:
                                        selling(winner_ticker, (tender_data.quantity - close_pos))
                                        close_pos = close_pos + (tender_data.quantity - close_pos)
                                        stop = True
                                except:
                                    if close_pos + unload_qty <= tender_data.quantity:
                                        selling(winner_ticker, unload_qty)
                                        close_pos = close_pos + unload_qty
                                        #sleep(sleeping)

                                    else:
                                        selling(winner_ticker, (tender_data.quantity - close_pos))
                                        close_pos = close_pos + (tender_data.quantity - close_pos)
                                        stop = True
                                    
                        else:
                            decline_tender(tender_data.tender_id)
                elif tender_data.is_fixed_bid == False:
                    decline_tender(tender_data.tender_id)
                    #if tick()+2 == tender_data.expires or tick()+1 == tender_data.expires:
                     #   if tender_data.action == 'SELL' or tender_data.action == 'OFFER':
                      #      price_auction = ticker_bid_ask(tender_data.ticker)[1] *1.05
                       # else:
                       #     price_auction = ticker_bid_ask(tender_data.ticker)[0] *0.95
                       # accept_tender_nofixed(tender_data.tender_id,price_auction)


